# Algorithm Implement

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
time_period = 15
class Q_Network(nn.Module):
    '''
    The input of this network should have shape (num_frame, 80, 80)
    '''

    def __init__(self, num_frame, num_action):
        super(Q_Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=num_frame, out_channels=32, kernel_size=(2,1), stride=1, padding=2)  # 16, 20, 20
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2,1), stride=1)  # 32, 9, 9
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(2,1), stride=1)  # 32, 9, 9
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(2,1), stride=1)  # 32, 9, 9
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(2,2), stride=1)  # 32, 9, 9
        self.pool = nn.AvgPool2d(kernel_size=(2,1))
        self.fc1 = nn.Linear(672, 256)
        self.fc2 = nn.Linear(256, num_action)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, image):
        x = F.relu(self.pool(self.conv1(image)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.pool(self.conv3(x)))
        x = x.view(-1, 672)
        x = F.relu(self.fc1(x))
        x1 = self.fc2(x)
        x1 = x1 - torch.max(x1, dim=1, keepdim=True)[0]
        x2 = self.fc3(x)
        return x1 + x2

In [ ]:
from torchsummary import summary
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Loading

In [ ]:
pwd

'/content'

In [ ]:
import pandas as pd
import pickle
import pandas as pd
import pickle


file = open('/content/drive/MyDrive/Model/stock_data_w_all_features_novelty_lag_sentiment', 'rb')
data = pickle.load(file)
file.close()



# codes = ['AAPL','AMZN','C','GOOG','JPM','NFLX','PLTR']
# for i in range(len(codes)):
#     data[i]['symbol'] = codes[i]

#df = pd.read_csv('/content/drive/MyDrive/TeamCapstone/concat_data.csv')
#df=df[['date', 'open', 'high', 'low', 'close', 'volume',
#       'positive', 'neutral', 'negative', 'tic']]
#data['date'] = [x[:10] for x in data['date']]
#df = data[(data['date']>='2022-01-01') & (data['date']<'2023-09-30')]
data.head()

,index,Open,High,Low,Close,Volume,Dividends,Stock Splits,returnsClosePrevRaw1,returnsOpenPrevRaw1,...,Novelty_0.5,Volume_0.5,Novelty_1,Volume_1,Novelty_3,Volume_3,Novelty_5,Volume_5,Novelty_7,Volume_7
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-18,0.0,169.572495,170.590858,167.496227,167.881821,90956700.0,0.0,0.0,-1.889419,0.099209,...,0.028156,811.0,0.023008,926.0,0.017296,1073.0,0.009907,1885.0,0.008128,3218.0
2022-01-18,2562.0,15.450000,15.750000,14.920000,14.970000,49067700.0,0.0,0.0,-6.495940,-3.074026,...,0.028156,811.0,0.023008,926.0,0.017296,1073.0,0.009907,1885.0,0.008128,3218.0
2022-01-18,854.0,61.282176,61.420007,59.821104,60.004890,26353600.0,0.0,0.0,-2.435381,2.034881,...,0.028156,811.0,0.023008,926.0,0.017296,1073.0,0.009907,1885.0,0.008128,3218.0
2022-01-18,2135.0,520.080017,521.750000,508.679993,510.799988,4839100.0,0.0,0.0,-2.832471,0.479143,...,0.028156,811.0,0.023008,926.0,0.017296,1073.0,0.009907,1885.0,0.008128,3218.0
2022-01-18,427.0,159.104996,159.734497,157.664505,158.917496,67292000.0,0.0,0.0,-1.986274,-0.652512,...,0.028156,811.0,0.023008,926.0,0.017296,1073.0,0.009907,1885.0,0.008128,3218.0


In [ ]:
data = data.reset_index()

In [ ]:
data.columns

Index(['Date', 'index', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
       'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
       'returnsOpenNextMktres10', 'next_10_day_relative_return',
       'next_day_return', 'next_day_relative_return', 'ticker', 'assetCode',
       'returnsClosePrevMktres10_lag_3_mean',
       'returnsClosePrevMktres10_lag_3_max',
       'returnsClosePrevMktres10_lag_3_min',
       'returnsClosePrevMktres10_lag_7_mean',
       'returnsClosePrevMktres10_lag_7_max',
       'returnsClosePrevMktres10_lag_7_min',
       'returnsClosePrevMktres10_lag_14_mean',
       'returnsClosePrevMktres10_lag_14_max',
       'returnsClosePrevMktres10_lag_14_min',
       'returnsClosePrevRaw10_lag_3_mean', 'returnsClosePrevRaw10_lag_3_max',
       'returnsClosePrevRaw10_lag_3_min', 

In [ ]:
### No need this cos date is already in proper format, use below if it is in this 'YYYY-MM-DD HH:MM:SS' format
##data['Date'] = [x[:10] for x in data['Date']]
from datetime import datetime
# Convert string to datetime.date
start_date = datetime.strptime('2022-01-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-09-30', '%Y-%m-%d').date()

df = data[(data['Date'] >= start_date) & (data['Date'] < end_date)]

#df = data[(data['Date']>='2022-01-01') & (data['Date']<'2023-09-30')]

In [ ]:
data['Date']

0       2022-01-18
1       2022-01-18
2       2022-01-18
3       2022-01-18
4       2022-01-18
           ...    
2984    2023-09-28
2985    2023-09-28
2986    2023-09-28
2987    2023-09-28
2988    2023-09-28
Name: Date, Length: 2989, dtype: object

In [ ]:
#data = pd.read_csv('/content/drive/MyDrive/TeamCapstone/min_data_adjust.csv')

In [ ]:

### selecting only AAPL
stock_df = data[data['ticker']=='AAPL']

In [ ]:
stock_df

,Date,index,Open,High,Low,Close,Volume,Dividends,Stock Splits,returnsClosePrevRaw1,...,Novelty_0.5,Volume_0.5,Novelty_1,Volume_1,Novelty_3,Volume_3,Novelty_5,Volume_5,Novelty_7,Volume_7
0,2022-01-18,0.0,169.572495,170.590858,167.496227,167.881821,90956700.0,0.0,0.0,-1.889419,...,0.028156,811.0,0.023008,926.0,0.017296,1073.0,0.009907,1885.0,0.008128,3218.0
13,2022-01-19,1.0,168.079558,169.147359,164.065425,164.352142,94815000.0,0.0,0.0,-2.102478,...,0.021503,839.0,0.013114,1063.0,0.010612,1830.0,0.008812,1991.0,0.007160,3351.0
15,2022-01-20,2.0,165.093679,167.763175,162.325307,162.651581,91420500.0,0.0,0.0,-1.034706,...,0.013233,4021.0,0.010449,4117.0,0.007932,4506.0,0.007650,4652.0,0.006624,5278.0
24,2022-01-21,3.0,162.562606,164.451033,160.466559,160.575317,122848900.0,0.0,0.0,-1.276510,...,0.011420,4902.0,0.007857,7877.0,0.006152,8900.0,0.005826,9157.0,0.005588,9382.0
33,2022-01-24,4.0,158.212319,160.466561,152.952410,159.794235,162294600.0,0.0,0.0,-0.486427,...,0.019466,1473.0,0.015964,1559.0,0.008813,4949.0,0.006242,10026.0,0.005585,10416.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2956,2023-09-22,422.0,174.440176,176.847008,173.820996,174.560013,56725400.0,0.0,0.0,0.494445,...,0.041174,535.0,0.025459,647.0,0.014182,1236.0,0.011638,1714.0,0.010868,1864.0
2966,2023-09-25,423.0,173.970796,176.737156,173.920859,175.848328,46172700.0,0.0,0.0,0.738035,...,0.041710,265.0,0.033900,320.0,0.017713,477.0,0.012190,1388.0,0.010324,1938.0
2972,2023-09-26,424.0,174.589987,174.969477,171.434141,171.733749,64588900.0,0.0,0.0,-2.339845,...,0.034391,746.0,0.023696,790.0,0.017446,1165.0,0.012740,1586.0,0.010509,2400.0
2978,2023-09-27,425.0,172.392870,172.812316,168.827576,170.205750,66921800.0,0.0,0.0,-0.889749,...,0.026180,755.0,0.016110,911.0,0.012062,1749.0,0.011105,1849.0,0.009364,2816.0


In [ ]:
stock_df.columns

Index(['Date', 'index', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
       'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
       'returnsOpenNextMktres10', 'next_10_day_relative_return',
       'next_day_return', 'next_day_relative_return', 'ticker', 'assetCode',
       'returnsClosePrevMktres10_lag_3_mean',
       'returnsClosePrevMktres10_lag_3_max',
       'returnsClosePrevMktres10_lag_3_min',
       'returnsClosePrevMktres10_lag_7_mean',
       'returnsClosePrevMktres10_lag_7_max',
       'returnsClosePrevMktres10_lag_7_min',
       'returnsClosePrevMktres10_lag_14_mean',
       'returnsClosePrevMktres10_lag_14_max',
       'returnsClosePrevMktres10_lag_14_min',
       'returnsClosePrevRaw10_lag_3_mean', 'returnsClosePrevRaw10_lag_3_max',
       'returnsClosePrevRaw10_lag_3_min', 

In [ ]:
columns_to_drop = [
    'Dividends', 'Stock Splits', 'length', 'returnsClosePrevRaw10',
    'returnsOpenPrevRaw10', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
    'returnsOpenNextMktres10', 'next_10_day_relative_return', 'Novelty_5', 'Volume_5',
    'Novelty_7', 'Volume_7', 'Close_lag_7_mean', 'Close_lag_7_max', 'Close_lag_7_min',
    'Close_lag_14_mean', 'Close_lag_14_max', 'Close_lag_14_min',
    'returnsClosePrevMktres10_lag_7_mean', 'returnsClosePrevMktres10_lag_7_max',
    'returnsClosePrevMktres10_lag_7_min', 'returnsClosePrevMktres10_lag_14_mean',
    'returnsClosePrevMktres10_lag_14_max', 'returnsClosePrevMktres10_lag_14_min',
    'returnsClosePrevRaw10_lag_7_mean', 'returnsClosePrevRaw10_lag_7_max',
    'returnsClosePrevRaw10_lag_7_min', 'returnsClosePrevRaw10_lag_14_mean',
    'returnsClosePrevRaw10_lag_14_max', 'returnsClosePrevRaw10_lag_14_min',
    'Open_lag_14_mean','Open_lag_14_max','Open_lag_14_min',
]


stock_df = stock_df.drop(columns=columns_to_drop)

In [ ]:
stock_df

,Date,index,Open,High,Low,Close,Volume,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,...,Bullish,To the Moon!!,Nay,Bearish,Novelty_0.5,Volume_0.5,Novelty_1,Volume_1,Novelty_3,Volume_3
0,2022-01-18,0.0,169.572495,170.590858,167.496227,167.881821,90956700.0,-1.889419,0.099209,-0.050625,...,1403.873047,-1986.760132,-906.276489,-2661.511963,0.028156,811.0,0.023008,926.0,0.017296,1073.0
13,2022-01-19,1.0,168.079558,169.147359,164.065425,164.352142,94815000.0,-2.102478,-0.880412,-1.133524,...,1903.575806,-2041.737061,-1189.204224,-3073.873047,0.021503,839.0,0.013114,1063.0,0.010612,1830.0
15,2022-01-20,2.0,165.093679,167.763175,162.325307,162.651581,91420500.0,-1.034706,-1.776467,0.069032,...,3655.196533,-6478.432129,-2176.565186,-7700.579590,0.013233,4021.0,0.010449,4117.0,0.007932,4506.0
24,2022-01-21,3.0,162.562606,164.451033,160.466559,160.575317,122848900.0,-1.276510,-1.533114,0.614972,...,4726.145508,-7055.808105,-2776.794434,-9011.958008,0.011420,4902.0,0.007857,7877.0,0.006152,8900.0
33,2022-01-24,4.0,158.212319,160.466561,152.952410,159.794235,162294600.0,-0.486427,-2.676069,-0.763601,...,2142.950195,-3645.181396,-1911.665527,-5043.228516,0.019466,1473.0,0.015964,1559.0,0.008813,4949.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2956,2023-09-22,422.0,174.440176,176.847008,173.820996,174.560013,56725400.0,0.494445,0.068738,0.724005,...,756.235840,-802.068298,-437.442932,-1699.409424,0.041174,535.0,0.025459,647.0,0.014182,1236.0
2966,2023-09-25,423.0,173.970796,176.737156,173.920859,175.848328,46172700.0,0.738035,-0.269077,0.335729,...,858.375305,-595.124573,-361.992950,-1645.980835,0.041710,265.0,0.033900,320.0,0.017713,477.0
2972,2023-09-26,424.0,174.589987,174.969477,171.434141,171.733749,64588900.0,-2.339845,0.355916,-0.866392,...,872.416138,-786.493103,-209.343750,-1805.105469,0.034391,746.0,0.023696,790.0,0.017446,1165.0
2978,2023-09-27,425.0,172.392870,172.812316,168.827576,170.205750,66921800.0,-0.889749,-1.258444,-0.912681,...,1311.664673,-1226.913696,-859.468506,-2490.690918,0.026180,755.0,0.016110,911.0,0.012062,1749.0


# Technical Indicators

In [ ]:
!pip install finta

In [ ]:
from finta import TA

In [ ]:
stock_df['SMA42'] = TA.SMA(stock_df, 42)
stock_df['SMA5'] = TA.SMA(stock_df, 5)
stock_df['SMA15'] = TA.SMA(stock_df, 15)
stock_df['AO'] = TA.AO(stock_df)
stock_df['OVB'] = TA.OBV(stock_df)
stock_df[['VW_MACD','MACD_SIGNAL']] = TA.VW_MACD(stock_df)
stock_df['RSI'] = TA.RSI(stock_df)
stock_df['CMO'] = TA.CMO(stock_df)

In [ ]:
stock_df = stock_df.dropna()

In [ ]:
stock_df.columns

Index(['Date', 'index', 'Open', 'High', 'Low', 'Close', 'Volume',
       'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'next_day_return',
       'next_day_relative_return', 'ticker', 'assetCode',
       'returnsClosePrevMktres10_lag_3_mean',
       'returnsClosePrevMktres10_lag_3_max',
       'returnsClosePrevMktres10_lag_3_min',
       'returnsClosePrevRaw10_lag_3_mean', 'returnsClosePrevRaw10_lag_3_max',
       'returnsClosePrevRaw10_lag_3_min', 'Open_lag_3_mean', 'Open_lag_3_max',
       'Open_lag_3_min', 'Open_lag_7_mean', 'Open_lag_7_max', 'Open_lag_7_min',
       'Close_lag_3_mean', 'Close_lag_3_max', 'Close_lag_3_min', 'Neutral',
       'Bullish', 'To the Moon!!', 'Nay', 'Bearish', 'Novelty_0.5',
       'Volume_0.5', 'Novelty_1', 'Volume_1', 'Novelty_3', 'Volume_3', 'SMA42',
       'SMA5', 'SMA15', 'AO', 'OVB', 'VW_MACD', 'MACD_SIGNAL', 'RSI', 'CMO'],
      dtype='object')

# Replay Buffer

In [ ]:
class SumTree:

    def __init__(self, capacity):

        self.capacity = capacity
        # the first capacity-1 positions are not leaves
        self.vals = [0 for _ in range(2*capacity - 1)] # think about why if you are not familiar with this

    def retrive(self, num):
        '''
        This function find the first index whose cumsum is no smaller than num
        '''
        ind = 0 # search from root
        while ind < self.capacity-1: # not a leaf
            left = 2*ind + 1
            right = left + 1
            if num > self.vals[left]: # the sum of the whole left tree is not large enouth
                num -= self.vals[left] # think about why?
                ind = right
            else: # search in the left tree
                ind = left
        return ind - self.capacity + 1

    def update(self, delta, ind):
        '''
        Change the value at ind by delta, and update the tree
        Notice that this ind should be the index in real memory part, instead of the ind in self.vals
        '''
        ind += self.capacity - 1
        while True:
            self.vals[ind] += delta
            if ind == 0:
                break
            ind -= 1
            ind //= 2

In [ ]:
# from collections import deque

# test = deque(maxlen=5)
# for i in range(10):
#     test.append(i)
#     print(test)

import numpy as np
import random
import bisect
import torch

ALPHA = 0.5
EPSILON = 0.05
TD_INIT = 1

class Replay_Buffer:
    '''
    Vanilla replay buffer
    '''

    def __init__(self, capacity=int(1e6), batch_size=None):

        self.capacity = capacity
        self.memory = [None for _ in range(capacity)] # save tuples (state, action, reward, next_state, done)
        self.ind_max = 0 # how many transitions have been stored

    def remember(self, state, action, reward, next_state, done):

        ind = self.ind_max % self.capacity
        self.memory[ind] = (state, action, reward, next_state, done)
        self.ind_max += 1

    def sample(self, k):
        '''
        return sampled transitions. Make sure that there are at least k transitions stored before calling this method
        '''
        index_set = random.sample(list(range(len(self))), k)
        states = torch.from_numpy(np.vstack([self.memory[ind][0] for ind in index_set])).float()
        actions = torch.from_numpy(np.vstack([self.memory[ind][1] for ind in index_set])).long()
        rewards = torch.from_numpy(np.vstack([self.memory[ind][2] for ind in index_set])).float()
        next_states = torch.from_numpy(np.vstack([self.memory[ind][3] for ind in index_set])).float()
        dones = torch.from_numpy(np.vstack([self.memory[ind][4] for ind in index_set]).astype(np.uint8)).float()

        return states, actions, rewards, next_states, dones

    def __len__(self):
        return min(self.ind_max, self.capacity)

class Rank_Replay_Buffer:
    '''
    Rank-based replay buffer
    '''

    def __init__(self, capacity=int(1e6), batch_size=64):
        self.capacity = capacity
        self.batch_size = batch_size
        self.alpha = ALPHA
        self.memory = [None for _ in range(capacity)]
        self.segments = [-1] + [None for _ in range(batch_size)] # the ith index will be in [segments[i-1]+1, segments[i]]

        self.errors = [] # saves (-TD_error, index of transition), sorted
        self.memory_to_rank = [None for _ in range(capacity)]

        self.ind_max = 0 # how many transitions have been stored
        self.total_weights = 0 # sum of p_i
        self.cumulated_weights = []

    def remember(self, state, action, reward, next_state, done):
        index = self.ind_max % self.capacity
        if self.ind_max >= self.capacity: # memory is full, need to pop
            self.pop(index)
        else: # memory is not full, need to adjust weights and find segment points
            self.total_weights += (1/(1+self.ind_max))**self.alpha # memory is not full, calculate new weights
            self.cumulated_weights.append(self.total_weights)
            self.update_segments()

        max_error = -self.errors[0][0] if self.errors else 0
        self.insert(max_error, index)
        self.memory[index] = (state, action, reward, next_state, done)
        self.ind_max += 1

    def sample(self, batch_size=None): # notive that batch_size is not used. It's just to unify the calling form
        index_set = [random.randint(self.segments[i]+1, self.segments[i+1]) for i in range(self.batch_size)]
        probs = torch.from_numpy(np.vstack([(1/(1+ind))**self.alpha/self.total_weights for ind in index_set])).float()

        index_set = [self.errors[ind][1] for ind in index_set]
        states = torch.from_numpy(np.vstack([self.memory[ind][0] for ind in index_set])).float()
        actions = torch.from_numpy(np.vstack([self.memory[ind][1] for ind in index_set])).long()
        rewards = torch.from_numpy(np.vstack([self.memory[ind][2] for ind in index_set])).float()
        next_states = torch.from_numpy(np.vstack([self.memory[ind][3] for ind in index_set])).float()
        dones = torch.from_numpy(np.vstack([self.memory[ind][4] for ind in index_set]).astype(np.uint8)).float()
        for ind in index_set:
            self.pop(ind)

        return index_set, states, actions, rewards, next_states, dones, probs

    def insert(self, error, index):
        '''
        Input :
            error : the TD-error of this transition
            index : the location of this transition
        insert error into self.errors, update self.memory_to_rank and self.rank_to_memory accordingly
        '''
        ind = bisect.bisect(self.errors, (-error, index))
        self.memory_to_rank[index] = ind
        self.errors.insert(ind, (-error, index))
        for i in range(ind+1, len(self.errors)):
            self.memory_to_rank[self.errors[i][1]] += 1

    def pop(self, index):
        '''
        Input :
            index : the location of a transition
        remove this transition, update self.memory_to_rank and self.rank_to_memory accordingly
        '''
        ind = self.memory_to_rank[index]
        self.memory_to_rank[index] = None
        self.errors.pop(ind)
        for i in range(ind, len(self.errors)):
            self.memory_to_rank[self.errors[i][1]] -= 1

    def update_segments(self):
        '''
        Update the segment points.
        '''
        if self.ind_max+1 < self.batch_size: # if there is no enough transitions
            return None
        for i in range(self.batch_size):
            ind = bisect.bisect_left(self.cumulated_weights, self.total_weights*((i+1)/self.batch_size))
            self.segments[i+1] = max(ind, self.segments[i]+1)

    def __len__(self):
        return min(self.capacity, self.ind_max)


class Proportion_Replay_Buffer:
    '''
    Proportion-based replay buffer
    '''

    def __init__(self, capacity=int(1e6), batch_size=None):
        self.capacity = capacity
        self.alpha = ALPHA
        self.memory = [None for _ in range(capacity)]
        self.weights = SumTree(self.capacity)
        self.default = TD_INIT
        self.ind_max = 0

    def remember(self, state, action, reward, next_state, done):
        index = self.ind_max % self.capacity
        self.memory[index] = (state, action, reward, next_state, done)
        delta = self.default+EPSILON - self.weights.vals[index+self.capacity-1]
        self.weights.update(delta, index)
        self.ind_max += 1

    def sample(self, batch_size):
        index_set = [self.weights.retrive(self.weights.vals[0]*random.random()) for _ in range(batch_size)]
        #print(index_set)
        probs = torch.from_numpy(np.vstack([self.weights.vals[ind+self.capacity-1]/self.weights.vals[0] for ind in index_set])).float()

        states = torch.from_numpy(np.vstack([self.memory[ind][0] for ind in index_set])).float()
        actions = torch.from_numpy(np.vstack([self.memory[ind][1] for ind in index_set])).long()
        rewards = torch.from_numpy(np.vstack([self.memory[ind][2] for ind in index_set])).float()
        next_states = torch.from_numpy(np.vstack([self.memory[ind][3] for ind in index_set])).float()
        dones = torch.from_numpy(np.vstack([self.memory[ind][4] for ind in index_set]).astype(np.uint8)).float()

        return index_set, states, actions, rewards, next_states, dones, probs

    def insert(self, error, index):
        delta = error+EPSILON - self.weights.vals[index+self.capacity-1]
        self.weights.update(delta, index)

    def __len__(self):
        return min(self.capacity, self.ind_max)

In [ ]:
tst = None
import random
from collections import deque
import torch
import torch.optim as optim
import numpy as np

# from networks import *

class Agent:

    def __init__(self, state_size, action_size, bs, lr, tau, gamma, device, visual=False):
        '''
        When dealing with visual inputs, state_size should work as num_of_frame
        '''
        self.state_size = state_size
        self.action_size = action_size
        self.bs = bs
        self.lr = lr
        self.tau = tau
        self.gamma = gamma
        self.device = device
        self.Q_local = Q_Network(self.state_size, self.action_size).to(device)
        self.Q_target = Q_Network(self.state_size, self.action_size).to(device)
        self.soft_update(1)
        self.optimizer = optim.Adam(self.Q_local.parameters(), self.lr)
        self.memory = Proportion_Replay_Buffer(int(1e5), bs)
        self.tst = None

    def act(self, state, eps=0):
        if random.random() > eps:
            state = torch.tensor(state, dtype=torch.float32).to(self.device)
            with torch.no_grad():
                action_values = self.Q_local(state)
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self):
        index_set, states, actions, rewards, next_states, dones, probs = self.memory.sample(self.bs)
        w = 1/len(self.memory)/probs
        w = w/torch.max(w)
        w = w.to(self.device)
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        Q_values = self.Q_local(states)
        Q_values = torch.gather(input=Q_values, dim=-1, index=actions)
        with torch.no_grad():
            Q_targets = self.Q_target(next_states)
            Q_targets, _ = torch.max(input=Q_targets, dim=-1, keepdim=True)
            Q_targets = rewards + self.gamma * (1 - dones) * Q_targets

        deltas = Q_values - Q_targets
        loss = (w*deltas.pow(2)).mean()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        deltas = np.abs(deltas.detach().cpu().numpy().reshape(-1))
        for i in range(self.bs):
            self.memory.insert(deltas[i], index_set[i])
    def soft_update(self, tau):
        for target_param, local_param in zip(self.Q_target.parameters(), self.Q_local.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

In [ ]:
Drop_list = ['Date', 'index', 'ticker', 'assetCode', 'Dividends',
 'Stock Splits', 'length', 'returnsClosePrevRaw10',
 'returnsOpenPrevRaw10',
 'returnsClosePrevMktres10',
 'returnsOpenPrevMktres10',
 'returnsOpenNextMktres10',
 'next_10_day_relative_return',
 'Novelty_5',
 'Volume_5',
 'Novelty_7',
 'Volume_7',
 'Close_lag_7_mean',
 'Close_lag_7_max',
 'Close_lag_7_min',
 'Close_lag_14_mean',
 'Close_lag_14_max',
 'Close_lag_14_min',

 'returnsClosePrevMktres10_lag_7_mean',
 'returnsClosePrevMktres10_lag_7_max',
 'returnsClosePrevMktres10_lag_7_min',
 'returnsClosePrevMktres10_lag_14_mean',
 'returnsClosePrevMktres10_lag_14_max',
 'returnsClosePrevMktres10_lag_14_min',
 'returnsClosePrevRaw10_lag_7_mean',
 'returnsClosePrevRaw10_lag_7_max',
 'returnsClosePrevRaw10_lag_7_min',
 'returnsClosePrevRaw10_lag_14_mean',
 'returnsClosePrevRaw10_lag_14_max',
 'returnsClosePrevRaw10_lag_14_min',
 'Open_lag_14_mean',
 'Open_lag_14_max',
 'Open_lag_14_min',
             ]

indicators = [col for col in stock_df.columns if col not in Drop_list]

indicators

['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'returnsClosePrevRaw1',
 'returnsOpenPrevRaw1',
 'returnsClosePrevMktres1',
 'returnsOpenPrevMktres1',
 'next_day_return',
 'next_day_relative_return',
 'returnsClosePrevMktres10_lag_3_mean',
 'returnsClosePrevMktres10_lag_3_max',
 'returnsClosePrevMktres10_lag_3_min',
 'returnsClosePrevRaw10_lag_3_mean',
 'returnsClosePrevRaw10_lag_3_max',
 'returnsClosePrevRaw10_lag_3_min',
 'Open_lag_3_mean',
 'Open_lag_3_max',
 'Open_lag_3_min',
 'Open_lag_7_mean',
 'Open_lag_7_max',
 'Open_lag_7_min',
 'Close_lag_3_mean',
 'Close_lag_3_max',
 'Close_lag_3_min',
 'Neutral',
 'Bullish',
 'To the Moon!!',
 'Nay',
 'Bearish',
 'Novelty_0.5',
 'Volume_0.5',
 'Novelty_1',
 'Volume_1',
 'Novelty_3',
 'Volume_3',
 'SMA42',
 'SMA5',
 'SMA15',
 'AO',
 'OVB',
 'VW_MACD',
 'MACD_SIGNAL',
 'RSI',
 'CMO']

In [ ]:

stock_df.columns
#indicators = ['Open', 'High', 'low', 'close', 'volume', 'positive', 'neutral', 'negative','SMA42', 'SMA5', 'SMA15', 'AO', 'OVB','VW_MACD',
#       'MACD_SIGNAL', 'RSI', 'CMO']

Index(['Date', 'index', 'Open', 'High', 'Low', 'Close', 'Volume',
       'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'next_day_return',
       'next_day_relative_return', 'ticker', 'assetCode',
       'returnsClosePrevMktres10_lag_3_mean',
       'returnsClosePrevMktres10_lag_3_max',
       'returnsClosePrevMktres10_lag_3_min',
       'returnsClosePrevRaw10_lag_3_mean', 'returnsClosePrevRaw10_lag_3_max',
       'returnsClosePrevRaw10_lag_3_min', 'Open_lag_3_mean', 'Open_lag_3_max',
       'Open_lag_3_min', 'Open_lag_7_mean', 'Open_lag_7_max', 'Open_lag_7_min',
       'Close_lag_3_mean', 'Close_lag_3_max', 'Close_lag_3_min', 'Neutral',
       'Bullish', 'To the Moon!!', 'Nay', 'Bearish', 'Novelty_0.5',
       'Volume_0.5', 'Novelty_1', 'Volume_1', 'Novelty_3', 'Volume_3', 'SMA42',
       'SMA5', 'SMA15', 'AO', 'OVB', 'VW_MACD', 'MACD_SIGNAL', 'RSI', 'CMO'],
      dtype='object')

In [ ]:
class Stock_Env:
    def __init__(self, initial_asset, data, cost):
        self.asset = initial_asset
        self.cash = initial_asset
        self.stock = 0
        self.data = data
        self.time = data.iloc[time_period]['Date']
        self.cost = cost
        self.history=[]
        self.total_cost = 0
        self.initial_asset = initial_asset
        self.rowid = time_period
        self.action_space = np.array(list(range(11)))

    def reset(self):
        self.asset = self.initial_asset
        self.cash = self.initial_asset
        self.stock = 0
        self.time = self.data.iloc[100]['Date']
        self.history=[]
        self.total_cost = 0
        self.rowid = time_period
        return self.data[:time_period][indicators].values

    def step(self, action):
        done = False
        states = self.data.iloc[self.rowid]
        self.rowid +=1
        if self.rowid == len(self.data)-1:
            done = True
        next_state = self.data.iloc[self.rowid]
        last_asset = self.asset
        price = next_state['Open']
        old_asset = self.cash + self.stock*price
        self.asset = old_asset
        target_value = action*0.1*self.asset
        distance = target_value - self.stock*price
        stock_distance = int(distance/(price*(1+self.cost)))
        self.stock += stock_distance
        self.cash = self.cash - distance - np.abs(stock_distance*self.cost*price)
        self.asset = self.cash+self.stock*price
        market_value = self.stock * next_state['Close']
        self.asset = market_value + self.cash
        reward = self.asset - last_asset
        self.time = next_state['Date']
        # self.stock = stock
        return (self.data[self.rowid-time_period:self.rowid][indicators].values, reward, done)

In [ ]:
#env = gym.make()
env = Stock_Env(1000000, stock_df, 0.002)
num_episode = 5
max_t = 1000
reward_log = []

for _ in range(num_episode):

    # initialize
    env.reset()
    t = 0
    episodic_reward = 0

    for t in range(max_t):

        #env.render()
        action = np.random.randint(11) # random action
        _, reward, done = env.step(action)
        episodic_reward += reward
        if done:
            break

    reward_log.append(episodic_reward)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = Agent(1, len(env.action_space), 64, 0.001, 0.001, 0.99, device, True)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#env = gym.make()
num_episode = 20000
max_t = 1000
reward_log = []
average_log = [] # monitor training process
eps = 1
eps_decay = 0.995
eps_min = 0.01
C = 4 # update weights every C steps

def train(env, agent, num_episode, eps_init, eps_decay, eps_min, max_t, num_frame=1, constant=0):
    rewards_log = []
    average_log = []
    eps = eps_init

    for i in range(1, 1 + num_episode):

        episodic_reward = 0
        done = False
        frame = env.reset()
        state_deque = deque(maxlen=num_frame)
        for _ in range(num_frame):
            state_deque.append(frame)
        state = np.stack(state_deque, axis=0)
        state = np.expand_dims(state, axis=0)
        t = 0

        while not done and t < max_t:

            t += 1
            action = agent.act(state, eps)
            frame, reward, done = env.step(action)
            state_deque.append(frame)
            next_state = np.stack(state_deque, axis=0)
            next_state = np.expand_dims(next_state, axis=0)
            agent.memory.memory.append((state, action, reward, next_state, done))

            if t % 5 == 0 and len(agent.memory) >= agent.bs:
                agent.learn()
                agent.soft_update(agent.tau)

            state = next_state.copy()
            episodic_reward += reward

        rewards_log.append(episodic_reward)
        average_log.append(np.mean(rewards_log[-100:]))
        print('\rEpisode {}, Reward {:.3f}, Average Reward {:.3f}'.format(i, episodic_reward, average_log[-1]), end='')
        if i % 100 == 0:
            print()

        eps = max(eps * eps_decay, eps_min)

    return rewards_log

In [ ]:
train(env, agent, num_episode, eps, eps_decay, eps_min, max_t, num_frame=1, constant=C)

Episode 1, Reward -146868.463, Average Reward -146868.463

RuntimeError: ignored